In [59]:
import requests
import bs4
from bs4 import BeautifulSoup as bs 
import re

import pandas as pd

url = 'https://www.who.int/tdr/grants/en/'

# define data columns
titles = []
providers = []
deadlines = []
descriptions = []

# clean up special characters
def clean(s):
  return re.sub('[^A-Za-z0-9 /-]+', '', s)

# requesting a session in the specific url
with requests.Session() as s: 
    r = s.get(url)
    src = r.content 
    soup = bs(src)
    soup.prettify()
    
    # strip contents
    for script in soup(['script','style']): 
        script.decompose()
    
    #finding grants
    grants_list = soup.find_all('div', attrs={'class':'inlay_color'})
    grants_list.pop() # remove irrelevant block
    #print( grants_list[0])

    
    for data in grants_list:
      soup = bs(str(data))
      
      text = str(soup.find('span'))
      # print(text)
      s_title = text.find('<span> ') + len('<span> ')
      e_title = text.find('<b>')
      s_ddl = text.find('<b>')+len('<b>')
      e_ddl = text.find('</b>')

      if s_title != -1 and e_title != -1 and s_ddl != -1 and e_ddl != -1: # layout type 1
        titles.append(clean(text[s_title:e_title]))
        deadlines.append(clean(text[s_ddl:e_ddl]))
        providers.append(soup.h4.string)
        link = 'https://www.who.int' + soup.li.a.attrs['href']
        descriptions.append(link)

      else: # layout type 2
        tlist = soup.find_all('h4')
        dlist = soup.find_all('h5')
        plist = soup.find_all('p')
        if len(tlist) > 1:
          for t,d,p in zip(tlist,dlist,plist):
            titles.append(t.string)
            deadlines.append(d.string)
            providers.append('')
            descriptions.append(p.string)

DataFrame = pd.DataFrame() 
DataFrame['Title']= titles 
DataFrame['Deadline']= deadlines
DataFrame['Provider'] = providers
DataFrame['Description'] = descriptions

Data = DataFrame.drop_duplicates() 

Data.to_csv("WHO_grants.csv")

print('Webscraping complete')



Webscraping complete
